# Koppeling en Verschonen van AAH Beleggingshypotheken met NNK Beleggingsfondsen

In [1]:
import re
import bs4
import time
import plyfile
import html5lib
import multiprocessing
import itertools

import numpy as np
import pandas as pd

import seaborn as sns
from scipy import misc
import scipy.io.wavfile as wavfile

import scipy
from math import sqrt
from scipy import stats
from pprint import pprint
from sklearn import tree
from sklearn.svm import SVC
from sklearn import manifold
from tempfile import mkdtemp
from textwrap import wrap
from matplotlib import cm as cm

import requests
from requests import auth

import sklearn.metrics as metrics
from pandas.plotting import scatter_matrix
from scipy.stats import randint as sp_randint
from sklearn.pipeline import TransformerMixin
from sklearn.metrics.scorer import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Binarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFECV, SelectFromModel, f_regression, SelectKBest
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor, AdaBoostClassifier
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Lasso, Ridge
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler, StandardScaler
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, roc_curve, roc_auc_score, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate, cross_val_predict, KFold, ShuffleSplit, StratifiedShuffleSplit

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm as cm
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import parallel_coordinates, andrews_curves

%matplotlib inline
matplotlib.style.use('ggplot')

pd.set_option('display.max_columns', None)

## Connectie maken met Marklogic

In [3]:
me = auth.HTTPDigestAuth("admin", "admin")
resp = requests.get("http://127.0.0.1:8002/manage/v2/hosts?format=json", auth=me)
print(resp.json())

{'host-default-list': {'meta': {'uri': '/manage/v2/hosts', 'current-time': '2018-02-08T11:35:19.914608+01:00', 'elapsed-time': {'units': 'sec', 'value': 0.845043}}, 'list-items': {'list-count': {'units': 'quantity', 'value': 1}, 'list-item': [{'uriref': '/manage/v2/hosts/macbook-pro-4.local', 'roleref': 'bootstrap', 'idref': '1796058674909374555', 'nameref': 'macbook-pro-4.local'}]}, 'related-views': {'related-view': [{'view-type': 'root', 'view-name': 'default', 'view-uri': '/manage/v2'}, {'view-type': 'list', 'view-name': 'metrics', 'view-uri': '/manage/v2/hosts?view=metrics'}, {'view-type': 'list', 'view-name': 'properties-schema', 'view-uri': '/manage/v2/hosts?view=properties-schema'}, {'view-type': 'list', 'view-name': 'schema', 'view-uri': '/manage/v2/hosts?view=schema'}, {'view-type': 'list', 'view-name': 'status', 'view-uri': '/manage/v2/hosts?view=status'}, {'view-type': 'list', 'view-name': 'health', 'view-uri': '/manage/v2?view=health&health-resource=host'}, {'view-type': 'l

## Documenten opvoeren en opvragen in Marklogic

In [4]:
obj = {'hypotheek':'ABN-AMRO', 'belegging':'NNK'};
obj

{'belegging': 'NNK', 'hypotheek': 'ABN-AMRO'}

In [6]:
one = '<one><child>The noble Brutus has told Caesar was ambitious</child></one>'
one

'<one><child>The noble Brutus has told Caesar was ambitious</child></one>'

In [5]:
two = {
  "two": {
    "child": "I come to bury Caesar, not to praise him."
  }
}
two

{'two': {'child': 'I come to bury Caesar, not to praise him.'}}

In [10]:
requests.post('http://localhost:8000/LATEST/documents?uri=/xml/one.xml', data = one, auth=me)

<Response [400]>

In [11]:
requests.post('http://localhost:8000/LATEST/documents?uri=/json/two.json', data = two, auth=me)

<Response [400]>

In [29]:
headers = {'Accept' : 'application/json'}

In [38]:
r_json = requests.get('http://localhost:8000/LATEST/search?q=caesar', headers=headers, auth=me)

In [40]:
r_json.json()

{'metrics': {'query-resolution-time': 'PT0.001951S',
  'snippet-resolution-time': 'PT0.000582S',
  'total-time': 'PT0.003212S'},
 'page-length': 10,
 'qtext': 'caesar',
 'results': [{'confidence': 0,
   'fitness': 0.2357023,
   'format': 'xml',
   'href': '/v1/documents?uri=%2Fxml%2Fone.xml',
   'index': 1,
   'matches': [{'match-text': ['The noble Brutus has told ',
      {'highlight': 'Caesar'},
      ' was ambitious'],
     'path': 'fn:doc("/xml/one.xml")/one/child'}],
   'mimetype': 'application/xml',
   'path': 'fn:doc("/xml/one.xml")',
   'score': 2048,
   'uri': '/xml/one.xml'}],
 'snippet-format': 'snippet',
 'start': 1,
 'total': 1}

In [47]:
r_doc = requests.get('http://localhost:8000/LATEST/documents?uri=/xml/one.xml', headers=headers, auth=me)

In [48]:
r_doc.text

'<?xml version="1.0" encoding="UTF-8"?>\n<one><child>The noble Brutus has told Caesar was ambitious</child></one>\n'